In [1]:
import requests
from time import sleep
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

URL = "https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=trade&Team=&BeginDate=2003-12-03&EndDate=2022-12-22&PlayerMovementChkBx=yes&Submit=Search"
URL = "https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=trade&Team=&BeginDate=2021-01-01&EndDate=&PlayerMovementChkBx=yes&Submit=Search"
#URL = "https://prosportstransactions.com/basketball/Search/SearchResults.php?Player=rights&Team=&BeginDate=2020-12-01&EndDate=&PlayerMovementChkBx=yes&Submit=Search"
Base = "https://prosportstransactions.com/basketball/Search/"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html5lib")

In [2]:
def update_edges_csv(file, filename, columns, data):
    df = pd.DataFrame([data], columns=columns)
    file = pd.concat([file, df])
    file.to_csv(filename, index=False)
    return file

In [3]:
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html5lib")
table = soup.find('table')
rows = table.findChildren(['th', 'tr'])

In [4]:
def classify_node(data: str):
    offset = 1 if data[0] == ' ' else 0
    if " cash" in data:
        return["C", "None"]
    elif "rights to" in data:
        return ["RT ", data[12+offset:]]
    elif "exception" in data:
        return ["EX ", "None"]
    elif "round pick" in data:
        if "#" in data and '-' in data and "?-?" not in data and "not exercised" not in data:
            return ["RPC ", data[data.index('-')+1:-1]]
        return ["RP ", "None"]
    else:
        if '•' in data:
            return ["PL ", data[data.index('•')+2:]]
        return ["UK ", "None"]
    

In [5]:
class node:
    def __init__(self, team, date):
        self.name = "n/a"
        self.node_type = "n/a"
        self.draft_details = "n/a"
        self.team = team
        self.date = date
        self.isRightsorPick = False
        self.isPlayer = False
        
    def classify_node(self, data: str):
        
        offset = 1 if data[0] == ' ' else 0
            
        if " cash" in data:
            self.node_type = "cash"
            
        elif "rights to" in data:
            self.name = data[12+offset:]
            self.node_type = "rights"
            self.isRightsorPick = True
            
        elif "exception" in data:
            self.node_type = "exception"
            
        elif "round pick" in data:
            if "#" in data and '-' in data and "?-?" not in data and "not exercised" not in data:
                self.name =  data[data.index('-')+1:-1]
                self.node_type = "conveyed"     
                self.draft_details = data[data.index('•')+2:]
                self.isRightsorPick = True
            else:
                self.node_type = "unconveyed"             
                self.draft_details = data[data.index('•')+2:]
                self.isRightsorPick = True

        else:
            if '•' in data:
                self.name = data[data.index('•')+2:]
                self.node_type = "player"
                self.isPlayer = True
            else:
                self.node_type = "undef"
    def print_data(self):
        print("node data -------------------------")
        if self.name != "n/a":
            print("name: " +  self.name)
        if self.node_type != "n/a":
            print("node_type: " +  self.node_type)
        if self.draft_details != "n/a":
            print("draft_details: " +  self.draft_details)
        print("team: " +  self.team)
        print("date: " +  self.date)
    
    

In [6]:
def get_teams_from_trade(team:str, notes: str):
    if notes[5] == "t":
        return(([team, notes[16:-5]]))
    elif notes[5] == "3":
        #print("********3 Team Trade********")
        temp = notes.replace(" ", "")
        return(([team] + temp[19:-5].split(',')))
    elif notes[5] == "4":
        #print("********4 Team Trade********")
        temp = notes.replace(" ", "")
        return(([team] + temp[19:-5].split(',')))
    elif notes[5] == "5":
        #print("********5 Team Trade********")
        temp = notes.replace(" ", "")
        return(([team] + temp[19:-5].split(',')))
    else:
        #print("********")
        return(["None", "None"])

In [7]:
def generate_visited_key(date: str, teams: list):
    temp = sorted(teams)
    res = [date] + temp
    return '_'.join(res)
    

In [8]:
def generate_edge_label(node1, node2):
    trade_details =  ["None", "None"]
    if node1.node_type == "cash":
        trade_details[0] = "Cash"
    elif node1.node_type == "rights":
        trade_details[0] = "Rights to " + node1.name
    elif node1.node_type == "conveyed" or node1.node_type == "conveyed":
        trade_details[0] = node1.draft_details
    elif node1.node_type == "exception":
        trade_details[0] = "Trade Exception"
    elif node1.node_type == "player":
        trade_details[0] = node1.name
        
    if node2.node_type == "cash":
        trade_details[1] = "Cash"
    elif node2.node_type == "rights":
        trade_details[1] = "Rights to " + node2.name
    elif node2.node_type == "conveyed" or node2.node_type == "conveyed":
        trade_details[1] = node2.draft_details
    elif node2.node_type == "exception":
        trade_details[1] = "Trade Exception"
    elif node2.node_type == "player":
        trade_details[1] = node2.name
        
    res = node1.team + " Aquire: " + trade_details[0] + " | " + node2.team + " Aquire: " + trade_details[1]
    return res
    

In [9]:
columns=['Key', 'Date', 'From', 'To', 'Label', 'Teams Involved', 'Player Trade', 'Rights/Pick Trade']
edges = pd.DataFrame(columns=columns)
edges.to_csv('edges.csv', index=False)
ID = 0
two_team_visited = set()
many_team_visited = set()
key_player_to_team = dict()
name_to_edges = dict()
edgekey_to_data = dict()

while URL != None:
#for _ in range(1):
    for row in rows:
        cells = row.findChildren('td')
        #print("ROW CONTENT-------------------------------------")
        date, team, acquired, relinq, notes = cells
        team = str(team)[5:-5]
        teams = get_teams_from_trade(team, str(notes))
        date = str(date)[20:-5]

        edge_key = [generate_visited_key(date, teams), None]
        nodes = [None, None]
        pl_involved = False
        rp_involved = False

        # Seperate two team and more than two team trades.
        # We only need to visit the first of the two team trade row
        if len(teams) == 2 and edge_key[0] not in two_team_visited:
            two_team_visited.add(edge_key[0])
            for acq in acquired:
                if acq and str(acq) != "<br/>":
                    nodes[0] = node(teams[0], date)
                    nodes[0].classify_node(str(acq))
                    for rel in relinq:
                        if rel and str(rel) != "<br/>":
                            nodes[1] = node(teams[1], date)
                            nodes[1].classify_node(str(rel))
                            temp = [min(nodes[0].name, nodes[1].name) , max(nodes[0].name, nodes[1].name)]
                            edge_key[1] = '+'.join(temp)
                            
                            if nodes[0].name != 'n/a' and nodes[1].name != 'n/a':
                                print("EDGE---------------------------")
                                EK = str('='.join(edge_key))
                                EL = generate_edge_label(nodes[0], nodes[1])
                                print("Edge Key: " + EK)
                                print(EL)
                                print(nodes[0].node_type, nodes[1].node_type)
                                pl_involved = nodes[0].isPlayer or nodes[1].isPlayer
                                rp_involved = nodes[0].isRightsorPick or nodes[1].isRightsorPick
                                edges = update_edges_csv(edges, 'edges.csv', columns, 
                                                         [EK, date, nodes[0].name, nodes[1].name, 
                                                          EL, str(teams), pl_involved, rp_involved])
                                name_to_edges[nodes[0].name] = name_to_edges.get(nodes[0].name, []) + [ID]
                                name_to_edges[nodes[1].name] = name_to_edges.get(nodes[1].name, []) + [ID]
                                ID += 1

        elif len(teams) > 2:
            data = [date, team, acquired, relinq, notes]
            #Only create nodes and edges of >2 team trades once we know where each player is coming from
            #Once we visit every one of the teams we have the suitable information
            if edge_key[0] not in edgekey_to_data or len(edgekey_to_data[edge_key[0]]) < len(teams):
                for acq in acquired:
                    curr = (classify_node(str(acq)))                    
                    if (curr[0] == 'PL ' or curr[0] == 'RPC ' or curr[0] == 'RT ') and curr[1] != 'None':
                        key_player_to_team[(edge_key[0], curr[1])] = team
                edgekey_to_data[edge_key[0]] = edgekey_to_data.get(edge_key[0], []) + [data]

            if len(edgekey_to_data[edge_key[0]]) == len(teams):
                teams = get_teams_from_trade(team, str(notes))
                for date, team, acquired, relinq, notes in edgekey_to_data[edge_key[0]]:
                    for acq in acquired:
                        if acq and str(acq) != "<br/>":
                            nodes[0] = node(team, date)
                            nodes[0].classify_node(str(acq))
                            for rel in relinq:
                                if rel and str(rel) != "<br/>":
                                    nodes[1] = node("Unknown", date)
                                    nodes[1].classify_node(str(rel))
                                       
                                    if (edge_key[0], nodes[1].name) in key_player_to_team:
                                        nodes[1].team = key_player_to_team[(edge_key[0], nodes[1].name)]
                                    temp = [min(nodes[0].name, nodes[1].name) , max(nodes[0].name, nodes[1].name)]
                                    edge_key[1] = '+'.join(temp)
                                    EK = str('='.join(edge_key))
                                    if nodes[0].name != "n/a" and nodes[1].name != "n/a" and (EK not in many_team_visited):
                                        
                                        #print("EDGE---------------------------")
                                        many_team_visited.add(EK)
                                        EL = generate_edge_label(nodes[0], nodes[1])
                                        print("Edge Key: " + EK)
                                        print(EL)
                                        pl_involved = nodes[0].isPlayer or nodes[1].isPlayer
                                        rp_involved = nodes[0].isRightsorPick or nodes[1].isRightsorPick
                                        edges = update_edges_csv(edges, 'edges.csv', columns, [EK, date, nodes[0].name, nodes[1].name,
                                                                                         EL, str(teams), pl_involved, rp_involved])
                                        name_to_edges[nodes[0].name] = name_to_edges.get(nodes[0].name, []) + [ID]
                                        name_to_edges[nodes[1].name] = name_to_edges.get(nodes[1].name, []) + [ID]
                                        ID += 1
    next_link = soup.find('div', class_='container').find_all('p')[-1].find('a')
    if next_link: 
        sleep(4)
        URL = "https://prosportstransactions.com/basketball/Search/"+next_link['href']
        page = requests.get(URL)
        soup = BeautifulSoup(page.content, "html5lib")
        table = soup.find('table')
        rows = table.findChildren(['th', 'tr'])
    else:
        URL = None
                

Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=Dante Exum+Jarrett Allen
Cavaliers Aquire: Jarrett Allen | Rockets Aquire: Dante Exum
Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=Jarrett Allen+MarJon Beauchamp
Cavaliers Aquire: Jarrett Allen | Rockets Aquire: 2022 first round pick (#24-MarJon Beauchamp)
Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=Dante Exum+Taurean Prince
Cavaliers Aquire: Taurean Prince | Rockets Aquire: Dante Exum
Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=MarJon Beauchamp+Taurean Prince
Cavaliers Aquire: Taurean Prince | Rockets Aquire: 2022 first round pick (#24-MarJon Beauchamp)
Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=Aleksandar Vezenkov+Dante Exum
Cavaliers Aquire: Rights to Aleksandar Vezenkov | Rockets Aquire: Dante Exum
Edge Key: 2021-01-16_Cavaliers_Nets_Pacers_Rockets=Aleksandar Vezenkov+MarJon Beauchamp
Cavaliers Aquire: Rights to Aleksandar Vezenkov | Rockets Aquire: 2022 first round pick (#24-MarJon Beauchamp)
Edge Key: 

EDGE---------------------------
Edge Key: 2021-03-25_Jazz_Raptors=David Johnson (Ricardo)+Matt Thomas
Jazz Aquire: Matt Thomas | Raptors Aquire: 2021 second round pick (#47-David Johnson (Ricardo))
player conveyed
EDGE---------------------------
Edge Key: 2021-06-18_Celtics_Thunder=Al Horford+Kemba Walker
Celtics Aquire: Al Horford | Thunder Aquire: Kemba Walker
player player
EDGE---------------------------
Edge Key: 2021-06-18_Celtics_Thunder=Al Horford+Alperen Sengun
Celtics Aquire: Al Horford | Thunder Aquire: 2021 first round pick (#16-Alperen Sengun)
player conveyed
EDGE---------------------------
Edge Key: 2021-06-18_Celtics_Thunder=Kemba Walker+Moses Brown
Celtics Aquire: Moses Brown | Thunder Aquire: Kemba Walker
player player
EDGE---------------------------
Edge Key: 2021-06-18_Celtics_Thunder=Alperen Sengun+Moses Brown
Celtics Aquire: Moses Brown | Thunder Aquire: 2021 first round pick (#16-Alperen Sengun)
player conveyed
EDGE---------------------------
Edge Key: 2021-07-30_B

EDGE---------------------------
Edge Key: 2021-08-08_Bulls_Pelicans=Garrett Temple+Lonzo Ball
Bulls Aquire: Lonzo Ball | Pelicans Aquire: Garrett Temple
player player
EDGE---------------------------
Edge Key: 2021-08-08_Bulls_Pelicans=Lonzo Ball+Tomas Satoransky
Bulls Aquire: Lonzo Ball | Pelicans Aquire: Tomas Satoransky
player player
EDGE---------------------------
Edge Key: 2021-08-11_Bulls_Spurs=DeMar DeRozan+Thaddeus Young
Bulls Aquire: DeMar DeRozan | Spurs Aquire: Thaddeus Young
player player
EDGE---------------------------
Edge Key: 2021-08-11_Bulls_Spurs=Al-Farouq Aminu+DeMar DeRozan
Bulls Aquire: DeMar DeRozan | Spurs Aquire: Al-Farouq Aminu
player player
EDGE---------------------------
Edge Key: 2021-08-11_Bulls_Spurs=DeMar DeRozan+Kennedy Chandler
Bulls Aquire: DeMar DeRozan | Spurs Aquire: 2022 second round pick (#38-Kennedy Chandler)
player conveyed
EDGE---------------------------
Edge Key: 2021-08-16_Clippers_Grizzlies=Eric Bledsoe+Patrick Beverley
Clippers Aquire: Eric 

Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Joe Ingles+Nickeil Alexander-Walker
Blazers Aquire: Joe Ingles | Jazz Aquire: Nickeil Alexander-Walker
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Joe Ingles+Tomas Satoransky
Blazers Aquire: Joe Ingles | Spurs Aquire: Tomas Satoransky
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Elijah Hughes+Nickeil Alexander-Walker
Blazers Aquire: Elijah Hughes | Jazz Aquire: Nickeil Alexander-Walker
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Elijah Hughes+Tomas Satoransky
Blazers Aquire: Elijah Hughes | Spurs Aquire: Tomas Satoransky
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Jabari Walker+Nickeil Alexander-Walker
Blazers Aquire: 2022 second round pick (from Jazz) (#59-Jabari Walker) | Jazz Aquire: Nickeil Alexander-Walker
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Jabari Walker+Tomas Satoransky
Blazers Aquire: 2022 second round pick (from Jazz) (#59-Jabari Walker) | Spurs Aquire: Tomas Satoransky
Edge Key: 2022-02-09_Blazers_Jazz_Spurs=Nickeil Alexander-Walker+Pacers
Jazz Aqui

EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=Ben Simmons+James Harden
76ers Aquire: James Harden | Nets Aquire: Ben Simmons
player player
EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=James Harden+Seth Curry
76ers Aquire: James Harden | Nets Aquire: Seth Curry
player player
EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=Andre Drummond+James Harden
76ers Aquire: James Harden | Nets Aquire: Andre Drummond
player player
EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=Ben Simmons+Paul Millsap
76ers Aquire: Paul Millsap | Nets Aquire: Ben Simmons
player player
EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=Paul Millsap+Seth Curry
76ers Aquire: Paul Millsap | Nets Aquire: Seth Curry
player player
EDGE---------------------------
Edge Key: 2022-02-10_76ers_Nets=Andre Drummond+Paul Millsap
76ers Aquire: Paul Millsap | Nets Aquire: Andre Drummond
player player
EDGE---------------------------
Edge Key: 2022-06-

In [10]:
nodes_df = pd.DataFrame(columns=['Name', 'Edges'])
for key in name_to_edges:
    df = pd.DataFrame([[key, str(name_to_edges[key])[1:-1]]], columns=['Name', 'Edges'])
    nodes_df = pd.concat([nodes_df, df])
nodes_df.to_csv('nodes.csv', index=False)
    